In [46]:
import pandas as pd
import ast
import requests
import time

## Reading saved data to python format

In [2]:
list_of_results = []

In [3]:
with open("results.txt", "r", encoding='utf-8') as file:
    for data in file.readlines():
        list_of_results.append(ast.literal_eval(data))

In [4]:
list_of_results

[('Warszawa, Białołęka, Nowodwory, Sprawna',
  '365 065 zł',
  ['Data dodania: 14 dni temu', 'Data aktualizacji: 6 dni temu'],
  [['Powierzchnia', '50,34 m²'],
   ['Liczba pokoi', '2'],
   ['Rynek', 'pierwotny'],
   ['Piętro', '1'],
   ['Liczba pięter', '5'],
   ['Rok budowy', '2020']],
  'https://www.otodom.pl/oferta/harmonogram-20-80-tarchomin-bezposredni-2020r-ID45Ys7.html#a2969c7ff8'),
 ('Warszawa, Bemowo, Apenińska',
  '450 000 zł',
  ['Data dodania: 9 dni temu', 'Data aktualizacji: 7 dni temu'],
  [['Powierzchnia', '46,70 m²'],
   ['Liczba pokoi', '3'],
   ['Rynek', 'wtórny'],
   ['Rodzaj zabudowy', 'blok'],
   ['Piętro', '2'],
   ['Liczba pięter', '3'],
   ['Ogrzewanie', 'miejskie'],
   ['Rok budowy', '1989'],
   ['Forma własności', 'pełna własność']],
  'https://www.otodom.pl/oferta/bemowo-po-generalnym-remoncie-ciche-ID462u3.html#8f74e2bbda'),
 ('Warszawa, Żoliborz',
  '420 000 zł',
  ['Data dodania: około 11 godzin temu',
   'Data aktualizacji: około 11 godzin temu'],
  [['Po

### Getting all possible parameters of properties

In [5]:
list_of_results_2 = [data[3] for data in list_of_results]
all_params = set([x[0] for y in list_of_results_2 for x in y])

### Creating final dictionary to pass it to pandas dataframe

In [6]:
final_dict = {}
final_dict["location"] = []
final_dict["total_price"] = []
final_dict["initial_date"] = []
final_dict["update_date"] = []
final_dict["link"] =[]
for param in all_params:
    final_dict[param] = []

In [7]:
for result in list_of_results:
    
    ## Getting location
    location = result[0]
    
    ## Getting total_price
    total_price = result[1]
    
    ## Getting dates
    initial_date = result[2][0]
    update_date = result[2][1]
    
    ## Getting link
    link = result[4]
    
    ## Saving above to dictionary 
    final_dict["location"].append(location)
    final_dict["total_price"].append(total_price)
    final_dict["initial_date"].append(initial_date)
    final_dict["update_date"].append(update_date)
    final_dict["link"].append(link)
    
    
    ## Getting params
    available_parameters = dict(result[3].copy())
    
    for param in all_params:
        
        if param in available_parameters.keys():
            final_dict[param].append(available_parameters[param])
        
        else:
            final_dict[param].append(None)
            
            
    
    

In [8]:
df = pd.DataFrame(final_dict)

## Adjusting dataframe

#### unpacking location

In [9]:
df_location = df.location.str.split(",", n = 4, expand = True)

In [10]:
df_location.columns = ["City", "District", "Detailed_location", "Additional_info", "Other"]

In [11]:
detailed_loc = df_location.apply(lambda x: x["Detailed_location"] if str(x["Additional_info"]) == "None" else x["Additional_info"], axis = 1)

In [12]:
df_location["Detailed_location_full"] = detailed_loc

### total_price

In [13]:
def extract_price(text):
    
    text = text.replace("zł", "")
    text = text.replace(" ", "")
    text = text.replace(",", ".")
    
    try:
        final_price = float(text)
    except:
        final_price = None
    
    return final_price

In [14]:
df["total_price"] = df.total_price.apply(lambda text: extract_price(text))
    

### Dealing with dates

In [15]:
df.loc[df["initial_date"].str.contains("minut|godzin|dzień|dni|mies|rok|lat")]

,location,total_price,initial_date,update_date,link,Czynsz,Forma własności,Materiał budynku,Piętro,Okna,Rodzaj zabudowy,Stan wykończenia,Liczba pięter,Rynek,Powierzchnia,Dostępne od,Liczba pokoi,Rok budowy,Ogrzewanie
0,"Warszawa, Białołęka, Nowodwory, Sprawna",365065.0,Data dodania: 14 dni temu,Data aktualizacji: 6 dni temu,https://www.otodom.pl/oferta/harmonogram-20-80...,None,None,None,1,None,None,None,5,pierwotny,"50,34 m²",None,2,2020,None
1,"Warszawa, Bemowo, Apenińska",450000.0,Data dodania: 9 dni temu,Data aktualizacji: 7 dni temu,https://www.otodom.pl/oferta/bemowo-po-general...,None,pełna własność,None,2,None,blok,None,3,wtórny,"46,70 m²",None,3,1989,miejskie
2,"Warszawa, Żoliborz",420000.0,Data dodania: około 11 godzin temu,Data aktualizacji: około 11 godzin temu,https://www.otodom.pl/oferta/zoliborz-34m2-2-p...,None,None,None,> 10,None,None,None,17,wtórny,34 m²,None,2,1967,None
3,"Warszawa, Żoliborz, Sady Żoliborskie",530000.0,Data dodania: 16 dni temu,Data aktualizacji: około 20 godzin temu,https://www.otodom.pl/oferta/mieszkanie-2-poko...,None,None,inne,3,None,blok,do zamieszkania,3,wtórny,"46,60 m²",None,2,1964,miejskie
4,"Warszawa, Mokotów, Górny Mokotów, al. Aleja Wi...",NaN,Data dodania: 15 dni temu,Data aktualizacji: 15 dni temu,https://www.otodom.pl/oferta/nowe-mieszkanie-2...,None,pełna własność,żelbet,5,plastikowe,apartamentowiec,do wykończenia,6,pierwotny,"53,27 m²",None,2,2021,miejskie
5,"Warszawa, Wola",735000.0,Data dodania: 14 dni temu,Data aktualizacji: 4 dni temu,https://www.otodom.pl/oferta/3-pokoje-przy-for...,530 zł,pełna własność,None,6,plastikowe,blok,do wykończenia,7,pierwotny,59 m²,None,3,2019,None
6,"Warszawa, Mokotów, Służewiec, Cybernetyki",630070.0,Data dodania: 13 dni temu,Data aktualizacji: 5 dni temu,https://www.otodom.pl/oferta/mieszkanie-trzypo...,None,None,inne,4,None,blok,do wykończenia,7,wtórny,"60,22 m²",None,3,2020,miejskie
7,"Warszawa, Białołęka, Dąbrówka Szlachecka, ul. ...",324000.0,Data dodania: 15 dni temu,Data aktualizacji: 13 dni temu,https://www.otodom.pl/oferta/ciche-i-przytulne...,500 zł,pełna własność,cegła,parter,plastikowe,blok,do zamieszkania,4,wtórny,39 m²,None,2,1999,miejskie
8,"Warszawa, Targówek, Targówek Mieszkaniowy, Mok...",495000.0,Data dodania: 3 dni temu,Data aktualizacji: dzień temu,https://www.otodom.pl/oferta/metro-trocka-3-po...,690 zł,pełna własność,wielka płyta,4,None,blok,do remontu,10,wtórny,64 m²,None,3,1975,miejskie
9,"Warszawa, Ursynów",820000.0,Data dodania: 11 dni temu,Data aktualizacji: 10 dni temu,https://www.otodom.pl/oferta/mieszkanie-76-5-m...,640 zł,spółdzielcze własnościowe,wielka płyta,9,plastikowe,blok,do wykończenia,12,wtórny,"76,50 m²",None,4,1985,miejskie


#### Możliwe warunki 
Czas liczony w dniach

* Minut i godzin -> Przekonwertowane na less than a day
* Dzień -> 1
* Dni (liczba - d) -> 1 * d
* Miesięcy (liczba - m) -> m * 30
* rok/lat (liczba - y) -> y * 365



In [16]:
import re

In [17]:
def date_extractor(x):
    
    
    x = x.replace("około ", "")
    x = x.replace("prawie ", "")
    x = x.replace("ponad ", "")
    
    if len(re.findall("minut|godzin", x)) > 0:
        return "Less than a day"
    
    elif "dzień" in x:
        return 1
    
    elif "dni " in x:
        x = x.replace("Data dodania: ", "")
        x = x.replace(" dni temu", "")
        
        return int(x)
    
    elif "miesiąc " in x:
        
        return 30
    
    elif any(m in x for m in ["miesiące", "miesięcy"]) == True:
        
        
        x = x.replace("Data dodania: ", "")
        x = x.replace(" miesiące temu", "")
        x = x.replace(" miesięcy temu", "")
        x = 30 * int(x)
        return x
    
    elif "rok" in x:
        return 365
    
    
     
    
    else:
        
#         x = x.replace("około ", "")
#         x = x.replace("ponad ", "")
        x = x.replace("Data dodania: ", "")
        x = x.replace(" lata temu", "")
        x = x.replace(" lat temu", "")
        
        x = 365 * int(x)
        
        return x
        

In [18]:
days_from_initial_announcement = df.initial_date.apply(lambda x: date_extractor(x))

### Dealing with powierzchnia

In [19]:
df["Powierzchnia"] = df["Powierzchnia"].apply(lambda x: float(x.split(" ")[0].replace(",", ".")))

### Dealing with czynsz

In [20]:
df["Czynsz"] = df["Czynsz"].apply(lambda x: float(x.replace(" zł", "").replace(" ", "").replace(",", ".")) if str(x) != 'None' else None)

### Final df

In [74]:
df_final = df_location.iloc[:,[1,5,4]].copy()

In [75]:
df_final["Total_price"] = df.loc[:,"total_price"]

In [76]:
df_final["Dates_from_initial_announcement"] = days_from_initial_announcement

In [77]:
df_final = pd.concat([df_final, df.iloc[:,5:]], axis = 1)

In [78]:
df_final = df_final.loc[df_final["Other"].isna() == True]

In [79]:
df_final = df_final.loc[df_final["Total_price"].isna() == False]

In [80]:
df_final.drop(columns = ["Other"], inplace = True)

In [81]:
df_final.reset_index(drop = True, inplace = True)

## Getting distance from city center (Pałac kultury i nauki)

#### First let's get coordinates of each location

In [29]:
headers = {
    "User-Agent": "MFHUSTLA420",
    "email": "grego23@onet.eu"
}
def get_coordinates(suburb, street, headers = headers):

    if street is not None:
        request = "https://nominatim.openstreetmap.org/search?suburb={}&street={}&city=Warsaw&country=pl&format=json&polygon=1&addressdetails=1&limit=1".format(suburb, street)
        
    else:
        request = "https://nominatim.openstreetmap.org/search?suburb={}&city=Warsaw&country=pl&format=json&polygon=1&addressdetails=1&limit=1".format(suburb)
    
    data = requests.get(request, headers = headers)
    data = data.json()
    
    data = data[0]
    
    lat, lon = (data["lat"], data["lon"])
    time.sleep(1.5)
    
    return((lat, lon))
    

In [30]:
list_of_coordinates = []
for i in range(0, len(df_final)):
    
    if i % 50 == 0:
        print(i)
    
    try:
        suburb = df_final.loc[i, "District"]
        street = df_final.loc[i, "Detailed_location_full"]
        lat, lon = get_coordinates(suburb, street)
        
    except:
        lat = None
        lon = None
    
    output = (i, lat, lon)
    
    list_of_coordinates.append(output)
    
    with open("coordinates.txt", "a", encoding = 'utf-8') as file:
        file.write(str(output))

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950
10000
10050


In [82]:
df_final = pd.concat([df_final, pd.DataFrame(list_of_coordinates, columns = ["ID", "Lat", "Lon"]).iloc[:,1:]], axis = 1)

In [83]:
df_final = df_final.loc[df_final["Lat"].isna() == False]
df_final.reset_index(inplace = True, drop = True)

In [236]:
pkin = get_coordinates("Śródmieście", "plac Defilad 1")

### Now let's get distance

In [43]:
def get_distance(lat, lon, headers = headers):
    
    ## We need to pass longitute and latituted of the location, second coordinates are constant for PKiN
    ## '52.2317641', '21.005799675616117'
    
    ## Generally we check the driving time
    request = 'http://router.project-osrm.org/route/v1/driving/{},{};21.005799675616117,52.2317641'.format(lon, lat)
    data = requests.get(request)
    data = data.json()
    
    ## Distance is in meters so we convert it to kms
    distance = data['routes'][0]['legs'][0]['distance']/1000
    
     ## Duration is in seconds, so convert to minutes
    time = data['routes'][0]['legs'][0]['duration']/60
    
    return ((distance, time))

In [279]:
test = get_distance(lat_test, lon_test)

In [50]:
import time as tm

In [58]:
list_of_distances = []
control = []
for i in range(0, len(df_final)):
    
    if i % 50 == 0:
        print(i)
    
    if df_final.loc[i, "Detailed_location_full"] is not None:
    
        lat = df_final.loc[i, "Lat"]
        lon = df_final.loc[i, "Lon"]

        try:
            distance, time = get_distance(lat, lon)
            control = []
        except:
            distance, time = (None, None)
            control.append("err")
    
    else:
        distance, time = (None, None)
        
    
    output = (i, distance, time)
    
    list_of_distances.append(output)
    
    with open("distances.txt", "a", encoding = 'utf-8') as file:
        file.write(str(output))
        
    tm.sleep(1)
    
    if len(control) == 20:
        print("too many empties")
        break 

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750


In [85]:
df_final = pd.concat([df_final, pd.DataFrame(list_of_distances, columns = ["ID", "Distance_to_PKIN_in_km", "Time_to_PKIN_in_minutes"]).iloc[:,1:]], axis =1)

### Kiedy są same dzielnice podane, to wtedy nie ma odległości i czasu. Więc zinputuję średnim czasem i odległością  

In [86]:
df_final

,District,Detailed_location_full,Total_price,Dates_from_initial_announcement,Czynsz,Forma własności,Materiał budynku,Piętro,Okna,Rodzaj zabudowy,...,Rynek,Powierzchnia,Dostępne od,Liczba pokoi,Rok budowy,Ogrzewanie,Lat,Lon,Distance_to_PKIN_in_km,Time_to_PKIN_in_minutes
0,Białołęka,Sprawna,365065.0,14,NaN,None,None,1,None,None,...,pierwotny,50.34,None,2,2020,None,52.3412347,20.9409258,16.2449,22.551667
1,Bemowo,Apenińska,450000.0,9,NaN,pełna własność,None,2,None,blok,...,wtórny,46.70,None,3,1989,miejskie,52.2534414,20.911715,8.4505,14.270000
2,Żoliborz,None,420000.0,Less than a day,NaN,None,None,> 10,None,None,...,wtórny,34.00,None,2,1967,None,52.2319581,21.0067249,NaN,NaN
3,Żoliborz,Sady Żoliborskie,530000.0,16,NaN,None,inne,3,None,blok,...,wtórny,46.60,None,2,1964,miejskie,52.2668547,20.9727739,5.1700,7.865000
4,Wola,None,735000.0,14,530.0,pełna własność,None,6,plastikowe,blok,...,pierwotny,59.00,None,3,2019,None,52.2319581,21.0067249,NaN,NaN
5,Mokotów,Cybernetyki,630070.0,13,NaN,None,inne,4,None,blok,...,wtórny,60.22,None,3,2020,miejskie,52.1755092,20.9979059,7.4484,10.970000
6,Białołęka,ul. Przylesie,324000.0,15,500.0,pełna własność,cegła,parter,plastikowe,blok,...,wtórny,39.00,None,2,1999,miejskie,52.3467073,20.9566364,15.9660,21.930000
7,Targówek,Mokra 4,495000.0,3,690.0,pełna własność,wielka płyta,4,None,blok,...,wtórny,64.00,None,3,1975,miejskie,52.251869049999996,21.202865677042325,18.4464,24.715000
8,Ursynów,None,820000.0,11,640.0,spółdzielcze własnościowe,wielka płyta,9,plastikowe,blok,...,wtórny,76.50,None,4,1985,miejskie,52.2319581,21.0067249,NaN,NaN
9,Ursus,ul. Aleksandra Prystora,499000.0,60,385.0,pełna własność,None,1,plastikowe,blok,...,wtórny,44.48,None,2,2017,miejskie,52.19021,20.9007185,9.9699,13.946667


In [87]:
df_final["Distance_to_PKIN_in_km"] = df_final.groupby('District')['Distance_to_PKIN_in_km'].apply(lambda x:x.fillna(x.mean()))

In [88]:
df_final["Time_to_PKIN_in_minutes"] = df_final.groupby('District')['Time_to_PKIN_in_minutes'].apply(lambda x:x.fillna(x.mean()))

In [93]:
df_final = df_final.loc[df_final["Distance_to_PKIN_in_km"].isna() == False]

In [94]:
df_final.reset_index(inplace = True, drop = True)

### Creating final version of dataframe

In [97]:
df_to_save = df_final.copy()

In [100]:
df_to_save.drop(columns = ["Detailed_location_full", "Dostępne od", "Okna", "Materiał budynku"], inplace = True)

In [105]:
df_to_save.rename(columns = {"Dates_from_initial_announcement": "Days_from_initial_announcement"}, inplace = True)

In [108]:
df_to_save.to_excel("df_final.xlsx", index = False)